# Calculate Cosine Similarity Score (KSA, Resume)

In [226]:
import pandas as pd
import numpy as np
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [227]:

Resume = pd.read_csv('/Users/dev/Desktop/Resumes_LIn.csv')


KSA = pd.read_excel('/Users/dev/Desktop/NICE Dataset.xlsx')

In [228]:
Resume.head(20)

,linkedinProfile,email,description,headline,location,firstName,lastName,fullName,connectionDegree,vmid,...,skill4,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,qualificationFromDropContact
0,https://www.linkedin.com/in/doug-andre-7230a85/,NaN,"Results-driven senior leader with a proven record of accomplishments leading information security initiatives in collaboration with key government cybersecurity partners to advance the security posture of organizations. Experienced building security programs that include development of plans and programs, identification of threats, analysis of risks, development of risk management plans, testing the effectiveness of security controls, reporting to senior management and oversight organizations on the status of information security. Organized, take-charge professional with exceptional follow-through abilities and detail orientation; able to plan and oversee programs from conception to successful operation.\n\nBroad, deep strategic knowledge with a rare and diverse background that spans both Commercial and Federal sectors of information security. Recognized for leadership and was elected to the Board of Directors for the International Information Systems Security Certification Consor...",Cyber at Cybersecurity and Infrastructure Security Agency,"Herndon, Virginia, United States",Doug,Andre,Doug Andre,3rd,ACoAAADnNAsBnmXqyWeLXCRg-Je4M77BB5Vb1w4,...,Information Security Management,99,Information Assurance,55,Computer Security,91,https://www.linkedin.com/in/doug-andre-7230a85/,doug-andre-7230a85,2023-03-07T02:07:02.760Z,NaN
1,https://www.linkedin.com/in/sematuncer/,sema.tuncer@skycep.com,Passionate CompTIA certified Cyber Security Analyst with strong background in analytical thinking and problem solving. Experienced in the incident response process and navigating challenging investigations using various security tools. Able to perform individually while thriving in a team environment with strong communication skills.,Cyber Security Analyst,Washington DC-Baltimore Area,Sema,Tuncer,Sema Tuncer,3rd,ACoAAD0h-tQBjQ1PNAt6Ghl0hd0sdGuY6n3LkoA,...,Forensic Analysis,5,Cybersecurity Incident Response,5,Cyber Threat Intelligence (CTI),5,https://www.linkedin.com/in/sematuncer/,sematuncer,2023-03-07T02:07:30.653Z,nominative@pro
2,https://www.linkedin.com/in/harsh-vanasiwala18/,NaN,"I am a student and a professional Cyber Threats Researcher currently with almost two year of experience under the domain of cyber security.\n\nThe temptation of the forbidden & knowing the unknowns has always given me the most satisfying feeling in the world. While simple observations, in addition to learning from my surroundings, have always been rewarding, my main magic box to those unanswered questions has led me to discover cyber security.\n\nI am a swift learner with adequate analytical skills and substantial Python knowledge. Excellent understanding of Cyber security and its applications. Apart from that, an excellent understanding of Network Mapper and Wire shark. Experience with a few python libraries like spacy, requests, beautifulsoup, selenium,, subprocess, socket and many more. Furthermore, I know about vulnerability scanners, open-source tools for Penetration Testing, Cyber security, and Digital Forensics. I can also automate assignments utilizing Python and currently ...",Cyber Security Enthusiast,"College Park, Maryland, United States",Harsh,Vanasiwala,Harsh Vanasiwala,3rd,ACoAADVYxJkBG7ARC66uy2PvSYYvw0uNqfvGlWo,...,C (Programming Language),9,Problem Solving,NaN,Linux,NaN,https://www.linkedin.com/in/harsh-vanasiwala18/,harsh-vanasiwala18,2023-03-07T02:07:58.575Z,NaN
3,https://www.linkedin.com/in/william-rollins-dc/,NaN,"Accomplished exploitation analyst bringing 8+ years of experience leading teams covering global \ncyber operations and managing red team campaigns / threat hunting operations across multiple agencies. Knowledgeable in planning, executing, and maintaining cyber oper

In [229]:
KSA

,OPM Code,NICE Specialty Area,NICE Specialty Area Description,Work Role,Work Role Description,KSA ID,KSA Description
0,511,Cybersecurity Defense Analysis (CDA),"Uses defensive measures and information collected from a variety of sources to identify, analyze, and report events that occur or might occur within the network to protect information, information systems, and networks from threats.",Cyber Defense Analyst,"Uses data collected from a variety of cyber defense tools (e.g., IDS alerts, firewalls, network traffic logs) to analyze events that occur within their environments for the purposes of mitigating threats.",A0010,Ability to analyze malware.
1,461,Systems Analysis (ANA),"Studies an organization's current computer systems and procedures, and designs information systems solutions to help the organization operate more securely, efficiently, and effectively. Brings business and information technology (IT) together by understanding the needs and limitations of both.",Systems Security Analyst,"Responsible for the analysis and development of the integration, testing, operations, and maintenance of systems security.",A0015,Ability to conduct vulnerability scans and recognize vulnerabilities in security systems.
2,511,Cybersecurity Defense Analysis (CDA),"Uses defensive measures and information collected from a variety of sources to identify, analyze, and report events that occur or might occur within the network to protect information, information systems, and networks from threats.",Cyber Defense Analyst,"Uses data collected from a variety of cyber defense tools (e.g., IDS alerts, firewalls, network traffic logs) to analyze events that occur within their environments for the purposes of mitigating threats.",A0066,"Ability to accurately and completely source all data used in intelligence, assessment and/or planning products."
3,531,Incident Response (CIR),"Responds to crises or urgent situations within the pertinent domain to mitigate immediate and potential threats. Uses mitigation, preparedness, and response and recovery approaches, as needed, to maximize survival of life, preservation of property, and information security. Investigates and analyzes all relevant response activities.",Cyber Defense Incident Responder,"Investigates, analyzes, and responds to cyber incidents within the network environment or enclave.",A0121,Ability to design incident response for cloud service models.
4,461,Systems Analysis (ANA),"Studies an organization's current computer systems and procedures, and designs information systems solutions to help the organization operate more securely, efficiently, and effectively. Brings business and information technology (IT) together by understanding the needs and limitations of both.",Systems Security Analyst,"Responsible for the analysis and development of the integration, testing, operations, and maintenance of systems security.",A0123,"Ability to apply cybersecurity and privacy principles to organizational requirements (relevant to confidentiality, integrity, availability, authentication, non-repudiation)."
...,...,...,...,...,...,...,...
124,511,Cybersecurity Defense Analysis (CDA),"Uses defensive measures and information collected from a variety of sources to identify, analyze, and report events that occur or might occur within the network to protect information, information systems, and networks from threats.",Cyber Defense Analyst,"Uses data collected from a variety of cyber defense tools (e.g., IDS alerts, firewalls, network traffic logs) to analyze events that occur within their environments for the purposes of mitigating threats.",S0169,Skill in conducting trend analysis.
125,531,Incident Response (CIR),"Responds to crises or urgent situations within the pertinent domain to mitigate immediate and potential threats. Uses mitigation, preparedness, and response and recovery approaches, as needed, to maximize survival of life, preservation of property, and information security. Investigates and analyz

In [230]:
Resume.columns

Index(['linkedinProfile', 'email', 'description', 'headline', 'location',
       'firstName', 'lastName', 'fullName', 'connectionDegree', 'vmid',
       'userId', 'linkedinSalesNavigatorUrl', 'connectionsCount',
       'connectionsUrl', 'mutualConnectionsUrl', 'company', 'companyUrl',
       'jobTitle', 'jobDescription', 'jobLocation', 'jobDateRange', 'company2',
       'companyUrl2', 'jobTitle2', 'jobDescription2', 'jobLocation2',
       'jobDateRange2', 'school', 'schoolUrl', 'schoolDegree',
       'schoolDescription', 'schoolDateRange', 'school2', 'schoolUrl2',
       'schoolDegree2', 'schoolDescription2', 'schoolDateRange2', 'allSkills',
       'skill1', 'endorsement1', 'skill2', 'endorsement2', 'skill3',
       'endorsement3', 'skill4', 'endorsement4', 'skill5', 'endorsement5',
       'skill6', 'endorsement6', 'baseUrl', 'profileId', 'timestamp',
       'qualificationFromDropContact'],
      dtype='object')

In [231]:
Resume.fillna('', inplace=True)

In [232]:
Resume['resume_content'] = Resume['description'] + Resume['headline'] + Resume['schoolDescription']  + Resume['qualificationFromDropContact'] + Resume['allSkills'] + Resume['skill1']+ Resume['skill4'] + Resume['skill5']+ Resume['skill6'] + Resume['skill2']+ Resume['skill3']+ Resume['jobDescription2'] +Resume['schoolDescription2']

In [233]:
Resume

,linkedinProfile,email,description,headline,location,firstName,lastName,fullName,connectionDegree,vmid,...,endorsement4,skill5,endorsement5,skill6,endorsement6,baseUrl,profileId,timestamp,qualificationFromDropContact,resume_content
0,https://www.linkedin.com/in/doug-andre-7230a85/,,"Results-driven senior leader with a proven record of accomplishments leading information security initiatives in collaboration with key government cybersecurity partners to advance the security posture of organizations. Experienced building security programs that include development of plans and programs, identification of threats, analysis of risks, development of risk management plans, testing the effectiveness of security controls, reporting to senior management and oversight organizations on the status of information security. Organized, take-charge professional with exceptional follow-through abilities and detail orientation; able to plan and oversee programs from conception to successful operation.\n\nBroad, deep strategic knowledge with a rare and diverse background that spans both Commercial and Federal sectors of information security. Recognized for leadership and was elected to the Board of Directors for the International Information Systems Security Certification Consor...",Cyber at Cybersecurity and Infrastructure Security Agency,"Herndon, Virginia, United States",Doug,Andre,Doug Andre,3rd,ACoAAADnNAsBnmXqyWeLXCRg-Je4M77BB5Vb1w4,...,99,Information Assurance,55,Computer Security,91,https://www.linkedin.com/in/doug-andre-7230a85/,doug-andre-7230a85,2023-03-07T02:07:02.760Z,,"Results-driven senior leader with a proven record of accomplishments leading information security initiatives in collaboration with key government cybersecurity partners to advance the security posture of organizations. Experienced building security programs that include development of plans and programs, identification of threats, analysis of risks, development of risk management plans, testing the effectiveness of security controls, reporting to senior management and oversight organizations on the status of information security. Organized, take-charge professional with exceptional follow-through abilities and detail orientation; able to plan and oversee programs from conception to successful operation.\n\nBroad, deep strategic knowledge with a rare and diverse background that spans both Commercial and Federal sectors of information security. Recognized for leadership and was elected to the Board of Directors for the International Information Systems Security Certification Consor..."
1,https://www.linkedin.com/in/sematuncer/,sema.tuncer@skycep.com,Passionate CompTIA certified Cyber Security Analyst with strong background in analytical thinking and problem solving. Experienced in the incident response process and navigating challenging investigations using various security tools. Able to perform individually while thriving in a team environment with strong communication skills.,Cyber Security Analyst,Washington DC-Baltimore Area,Sema,Tuncer,Sema Tuncer,3rd,ACoAAD0h-tQBjQ1PNAt6Ghl0hd0sdGuY6n3LkoA,...,5,Cybersecurity Incident Response,5,Cyber Threat Intelligence (CTI),5,https://www.linkedin.com/in/sematuncer/,sematuncer,2023-03-07T02:07:30.653Z,nominative@pro,"Passionate CompTIA certified Cyber Security Analyst with strong background in analytical thinking and problem solving. Experienced in the incident response process and navigating challenging investigations using various security tools. Able to perform individually while thriving in a team environment with strong communication skills.Cyber Security AnalystSkills: Business Development ¬∑ SQLnominative@proThreat & Vulnerability Management, PagerDuty, Forensic Toolkit (FTK), Forensic Analysis, Cybersecurity Incident Response, Cyber Threat Intelligence (CTI), Detective Security Control at Crosstrike, Mitigation and Response Insights, SentinelOne, Investigating Alerts at Crowdstrike, Creating Custom IOA at Crowstrike, P

In [234]:
Resume['resume_content'] 

0      Results-driven senior leader with a proven record of accomplishments leading information security initiatives in collaboration with key government cybersecurity partners to advance the security posture of organizations. Experienced building security programs that include development of plans and programs, identification of threats, analysis of risks, development of risk management plans, testing the effectiveness of security controls, reporting to senior management and oversight organizations on the status of information security. Organized, take-charge professional with exceptional follow-through abilities and detail orientation; able to plan and oversee programs from conception to successful operation.\n\nBroad, deep strategic knowledge with a rare and diverse background that spans both Commercial and Federal sectors of information security.  Recognized for leadership and was elected to the Board of Directors for the International Information Systems Security Certification Con

In [235]:

def preprocess_text(text):
    # Remove punctuation and convert to lowercase
    text = re.sub(r'[^\w\s]', '', str(text).lower())
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    # Stem the words
    stemmer = PorterStemmer()
    text = ' '.join([stemmer.stem(word) for word in text.split()])
    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split()])
    return text



Resume['resume_content'] = Resume['resume_content'].apply(preprocess_text)
KSA['KSA Description'] = KSA['KSA Description'].apply(preprocess_text)

In [236]:
Resume['resume_content']

0      resultsdriven senior leader proven record accomplish lead inform secur initi collabor key govern cybersecur partner advanc secur postur organ experienc build secur program includ develop plan program identif threat analysi risk develop risk manag plan test effect secur control report senior manag oversight organ statu inform secur organ takecharg profession except followthrough abil detail orient abl plan overse program concept success oper broad deep strateg knowledg rare diver background span commerci feder sector inform secur recogn leadership elect board director intern inform system secur certif consortium inc isc global notforprofit leader educ certifi inform secur profession throughout career addit select member white hous secur metric task forc defin new feder inform secur manag act fisma perform outcomebas metric inform secur perform metric highlight risk area need improv feder agenc repres signific shift prior fisma reportingcyb cybersecur infrastructur secur agencycis

In [237]:
KSA['KSA Description']

0                                                                                          abil analyz malwar
1                                                         abil conduct vulner scan recogn vulner secur system
2                                           abil accur complet sourc data use intellig ass andor plan product
3                                                                abil design incid respons cloud servic model
4       abil appli cybersecur privaci principl organiz requir relev confidenti integr avail authent nonrepudi
                                                        ...                                                  
124                                                                               skill conduct trend analysi
125                                                                         skill use secur event correl tool
126                                                             skill design incid respons cloud servic model
127    ski

In [238]:

KSA_string = ' '.join(KSA['KSA Description'])


KSA_df = pd.DataFrame({'KSA': KSA_string}, index=[0])


KSA_df

,KSA
0,abil analyz malwar abil conduct vulner scan recogn vulner secur system abil accur complet sourc data use intellig ass andor plan product abil design incid respons cloud servic model abil appli cybersecur privaci principl organiz requir relev confidenti integr avail authent nonrepudi abil appli techniqu detect host networkbas intrus use intrus detect technolog abil interpret inform collect network tool eg nslookup ping tracerout knowledg comput network concept protocol network secur methodolog knowledg risk manag process eg method ass mitig risk knowledg law regul polici ethic relat cybersecur privaci knowledg cybersecur privaci principl knowledg cyber threat vulner knowledg specif oper impact cybersecur lap knowledg authent author access control method knowledg cyber defens vulner ass tool capabl knowledg comput algorithm knowledg encrypt algorithm knowledg cryptographi cryptograph key manag concept knowledg data backup recoveri knowledg databas system knowledg busi continu disast ...


In [239]:
documents = list(KSA['KSA Description'].values)

In [240]:
ksa_text = ' '.join(KSA['KSA Description'].fillna(''))

# create a CountVectorizer object
# CountVectorizer

cv = CountVectorizer()

# fit and transform the text in ksa_text using CountVectorizer
X = cv.fit_transform([ksa_text])

# calculate the cosine similarity between the ksa_text and the resume_content
cos_sim = cosine_similarity(X, cv.transform(Resume['resume_content'].fillna('')))

# store the cosine similarity values in a new column in the Resume DataFrame
Resume['cosCountvect'] = cos_sim[0]


In [241]:

print(Resume.loc[:, ['firstName', 'lastName','cosCountvect']])

           firstName       lastName  cosCountvect
0               Doug          Andre      0.346833
1               Sema         Tuncer      0.341418
2              Harsh     Vanasiwala      0.358870
3            William        Rollins      0.328418
4              Tamir  Orbach (CISM)      0.342423
..               ...            ...           ...
710             Tyan         Morgan      0.173794
711           Collin           Silk      0.120363
712           Thomas    Howser, CPA      0.068698
713  Siyuan (Sydnie)             S.      0.015266
714            Kelly            Sim      0.174912

[715 rows x 3 columns]


In [242]:

print(Resume['cosCountvect'].describe())

count    715.000000
mean       0.146708
std        0.083429
min        0.000000
25%        0.092188
50%        0.137399
75%        0.191213
max        0.449514
Name: cosCountvect, dtype: float64


In [243]:
Resume['schoolDegree2']

0                                                   B.S., Criminal Justice
1      Healthcare Management, Health/Health Care Administration/Management
2                                  Bachelor's degree, Computer Engineering
3                               Bachelor of Science - BS, Computer Science
4                                             BSEE, Electrical Engineering
                                      ...                                 
710             Bachelor of Science in Business Administration, Accounting
711                                                                       
712                                                                       
713                                                 Bachelor of Accounting
714                            Graduate Certificate in Forensic Accounting
Name: schoolDegree2, Length: 715, dtype: object

In [244]:
Resume['schoolDegree']

0                                                                 Masters, Information Security
1      Master of Business Administration - MBA, Business Administration and Management, General
2                                                  Master of Engineering - MEng, Cyber Security
3                     Master of Science - MS, Computer and Information Sciences, Data Analytics
4                                                                  MSEE, Electrical Engineering
                                                 ...                                           
710                                                                 Master's degree, Accounting
711                                                         Bachelor's degree, Finance, General
712                                                               Bachelor's degree, Accounting
713                                                                 Master's degree, Accounting
714                                     

In [245]:
filtered_resume = Resume[Resume['cosCountvect'] > 0.2]

In [246]:
a = filtered_resume.loc[:,['firstName','lastName','schoolDegree','schoolDegree2','location']]

In [247]:
a

,firstName,lastName,schoolDegree,schoolDegree2,location
0,Doug,Andre,"Masters, Information Security","B.S., Criminal Justice","Herndon, Virginia, United States"
1,Sema,Tuncer,"Master of Business Administration - MBA, Business Administration and Management, General","Healthcare Management, Health/Health Care Administration/Management",Washington DC-Baltimore Area
2,Harsh,Vanasiwala,"Master of Engineering - MEng, Cyber Security","Bachelor's degree, Computer Engineering","College Park, Maryland, United States"
3,William,Rollins,"Master of Science - MS, Computer and Information Sciences, Data Analytics","Bachelor of Science - BS, Computer Science","Washington, District of Columbia, United States"
4,Tamir,Orbach (CISM),"MSEE, Electrical Engineering","BSEE, Electrical Engineering",Washington DC-Baltimore Area
...,...,...,...,...,...
693,Terri-Anne,"Lowe FCCA, CA, CIA",,"Bachelor of Science B.Sc (Hons) , Accounting","Arlington, Virginia, United States"
699,James,Jaskolski,"Bachelor of Arts (B.A.), Economics",,"Gaithersburg, Maryland, United States"
701,Abesha,Ayalew,"Certification, Cybersecurity",,Washington DC-Baltimore Area
708,Michelle,Castillo,"Bachelor of Science - BS, Accounting",High School Diploma,"Gaithersburg, Maryland, United States"


In [248]:
import re

# define a function to replace 'Master of' with 'MS'
def replace_master_of_with_ms(text):
    text = re.sub(r'Masters (\w+)', r'MS, \1', text)
    return text
def replace_master_of_with_bs(text):
    text = re.sub(r'Bachelor’s(\w+)', r'BS, \1', text)
    return text
# apply the function to the 'schoolDegree' column
a['schoolDegree'] = a['schoolDegree'].apply(replace_master_of_with_ms)

a['schoolDegree'] = a['schoolDegree'].apply(replace_master_of_with_bs)

In [249]:
for i, row in a.iterrows():
    # check if the column value contains one or two instances of 'MS'
    if row['schoolDegree'].count('MS') == 1:
        # replace 'Master of' with 'MS'
        a.loc[i, 'schoolDegree'] = row['schoolDegree'].replace('Master of', 'MS', 1)
    elif row['schoolDegree'].count('MS') == 2:
        # remove the substring until the first comma
        a.loc[i, 'schoolDegree'] = row['schoolDegree'].split(',', 1)[1].strip()


In [250]:
a

,firstName,lastName,schoolDegree,schoolDegree2,location
0,Doug,Andre,"Masters, Information Security","B.S., Criminal Justice","Herndon, Virginia, United States"
1,Sema,Tuncer,"Master of Business Administration - MBA, Business Administration and Management, General","Healthcare Management, Health/Health Care Administration/Management",Washington DC-Baltimore Area
2,Harsh,Vanasiwala,"Master of Engineering - MEng, Cyber Security","Bachelor's degree, Computer Engineering","College Park, Maryland, United States"
3,William,Rollins,"MS Science - MS, Computer and Information Sciences, Data Analytics","Bachelor of Science - BS, Computer Science","Washington, District of Columbia, United States"
4,Tamir,Orbach (CISM),"MSEE, Electrical Engineering","BSEE, Electrical Engineering",Washington DC-Baltimore Area
...,...,...,...,...,...
693,Terri-Anne,"Lowe FCCA, CA, CIA",,"Bachelor of Science B.Sc (Hons) , Accounting","Arlington, Virginia, United States"
699,James,Jaskolski,"Bachelor of Arts (B.A.), Economics",,"Gaithersburg, Maryland, United States"
701,Abesha,Ayalew,"Certification, Cybersecurity",,Washington DC-Baltimore Area
708,Michelle,Castillo,"Bachelor of Science - BS, Accounting",High School Diploma,"Gaithersburg, Maryland, United States"


In [251]:
import pandas as pd



# replace 'Masters of' with 'MS'
a['schoolDegree'] = a['schoolDegree'].str.replace('Masters of', 'MS', case=False)

# replace 'Master of' with 'MS' only if it occurs twice in the string
a['schoolDegree'] = a['schoolDegree'].apply(lambda x: 'MS' + x.split('Master of', 2)[-1] if x.count('Master of') == 2 else x)

# replace 'MBA' with 'MS'
a['schoolDegree'] = a['schoolDegree'].str.replace('MBA', 'MS', case=False)

print(a)


      firstName             lastName  \
0          Doug                Andre   
1          Sema               Tuncer   
2         Harsh           Vanasiwala   
3       William              Rollins   
4         Tamir        Orbach (CISM)   
..          ...                  ...   
693  Terri-Anne  Lowe FCCA, CA, CIA    
699       James            Jaskolski   
701      Abesha               Ayalew   
708    Michelle             Castillo   
709         Soh               Samuto   

                                                                                schoolDegree  \
0                                                              Masters, Information Security   
1    Master of Business Administration - MS, Business Administration and Management, General   
2                                               Master of Engineering - MEng, Cyber Security   
3                         MS Science - MS, Computer and Information Sciences, Data Analytics   
4                                      

In [252]:
import re

def degree_abbrv(s):
    s = str(s)
    if 'MS' in s:
        if s.count('MS') == 2:
            s = re.sub('^.*?,\s*', '', s)
        else:
            s = s.replace('Master of Science', 'MS')
    elif 'BS' in s:
        s = re.sub('^.*?,\s*', '', s)
        s = s.replace('Bachelor of Science', 'BS').replace("Bachelor's degree", 'BS').replace('Bachelor of Engineering', 'BE').replace('Bachelor of Applied Science', 'BASc')
    return s


In [253]:
a['schoolDegree'] = a['schoolDegree'].apply(degree_abbrv)


In [254]:
a

,firstName,lastName,schoolDegree,schoolDegree2,location
0,Doug,Andre,"Masters, Information Security","B.S., Criminal Justice","Herndon, Virginia, United States"
1,Sema,Tuncer,"Master of Business Administration - MS, Business Administration and Management, General","Healthcare Management, Health/Health Care Administration/Management",Washington DC-Baltimore Area
2,Harsh,Vanasiwala,"Master of Engineering - MEng, Cyber Security","Bachelor's degree, Computer Engineering","College Park, Maryland, United States"
3,William,Rollins,"Computer and Information Sciences, Data Analytics","Bachelor of Science - BS, Computer Science","Washington, District of Columbia, United States"
4,Tamir,Orbach (CISM),"MSEE, Electrical Engineering","BSEE, Electrical Engineering",Washington DC-Baltimore Area
...,...,...,...,...,...
693,Terri-Anne,"Lowe FCCA, CA, CIA",,"Bachelor of Science B.Sc (Hons) , Accounting","Arlington, Virginia, United States"
699,James,Jaskolski,"Bachelor of Arts (B.A.), Economics",,"Gaithersburg, Maryland, United States"
701,Abesha,Ayalew,"Certification, Cybersecurity",,Washington DC-Baltimore Area
708,Michelle,Castillo,Accounting,High School Diploma,"Gaithersburg, Maryland, United States"


In [255]:
pd.options.display.max_colwidth = 1000

# display the entire content of the column 'my_column'
print(a)

      firstName             lastName  \
0          Doug                Andre   
1          Sema               Tuncer   
2         Harsh           Vanasiwala   
3       William              Rollins   
4         Tamir        Orbach (CISM)   
..          ...                  ...   
693  Terri-Anne  Lowe FCCA, CA, CIA    
699       James            Jaskolski   
701      Abesha               Ayalew   
708    Michelle             Castillo   
709         Soh               Samuto   

                                                                                schoolDegree  \
0                                                              Masters, Information Security   
1    Master of Business Administration - MS, Business Administration and Management, General   
2                                               Master of Engineering - MEng, Cyber Security   
3                                          Computer and Information Sciences, Data Analytics   
4                                      

In [259]:
a = a[a["schoolDegree"].str.contains("Engineering|Technology|Information|Cyber|System|Security|Computer|University|Data|Analytics|Science") == False]
result = a[a["schoolDegree2"].str.contains("Engineering|Technology|Information|Cyber|System|Security|Computer|University|Data|Analytics|Science") == False]



In [260]:
print(result)

     firstName                 lastName  \
1         Sema                   Tuncer   
25       Kevin  Mornot CISM, CASP, GCIH   
71      Hannah                       W.   
76        Jade                     Soto   
104    Miracle            Heagle , B.S.   
..         ...                      ...   
683      Paola       Trevino , CPA, MBA   
687  Peter Rey              Ikundi, MBA   
699      James                Jaskolski   
708   Michelle                 Castillo   
709        Soh                   Samuto   

                                                                                     schoolDegree  \
1         Master of Business Administration - MS, Business Administration and Management, General   
25                                                                                                  
71                                                              Bachelor of Arts - BA, Psychology   
76                                                                             

In [261]:
result.to_csv('profiles_extracted.csv', header = True)